This notebook explores different dosage ratios for a system that's symmetric except for diffusion coefficients. I explore diffusion coefficients of $D_A = 0.1$/$D_P=1.1$ as well as $D_A = 0$/$D_P=1.1$. There is a quantitative difference, but qualitatively nothing changes.


In [ ]:
import bisect
import copy
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
from scipy import optimize
import sys
import time
from IPython.core.debugger import Tracer
# Profiling
import cProfile
import re
sys.path.append(os.path.abspath('../')+'/PythonDet/')
from ParDetAsymm import ParSim, Sim_Container, s_to_v
# from ParDetAsymmO import ParSim as ParSimO

In [ ]:
%qtconsole

Symmetric system, only thing varied is dosage ratio

The method to find the border between regions of stability and instability relies on bisection: e.g. try one ratio for a given size and diffusion coefficient, check whether A or P dominates or the system is polarized. Then move on to the next simulation by interval halving. This way, the border is approached from two sides.

In [ ]:
ratio_Adom = []
size_Adom = []
ratio_P = []
size_P = []
for file in glob.glob("/Volumes/Transcend/PARmodellingData/pickled_FindA041217/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        ratio_Adom.append(p.ratio)
        size_Adom.append(p.sys_size)
    elif (p.finished_in_time == 0) or (p.finished_in_time == 2):
        ratio_P.append(p.ratio)
        size_P.append(p.sys_size)

In [ ]:
ratio_Pdom = []
size_Pdom = []
ratio_A = []
size_A = []
for file in glob.glob("/Volumes/Transcend/PARmodellingData/pickled_FindP041217/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 0):
        ratio_Pdom.append(p.ratio)
        size_Pdom.append(p.sys_size)
    elif (p.finished_in_time == 1) or (p.finished_in_time == 2):
        ratio_A.append(p.ratio)
        size_A.append(p.sys_size)

In [ ]:
fig=plt.figure(figsize=(8, 7), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(ratio_Adom, size_Adom, 'r.')
plt.plot(ratio_P, size_P, 'm.')
plt.plot(ratio_Pdom, size_Pdom, 'b.')
plt.plot(ratio_A, size_A, 'c.')
plt.show()
plt.show()

In [ ]:
fig=plt.figure(figsize=(8, 7), dpi= 80, facecolor='w', edgecolor='k')
# Collect all ratios/sizes in dataframes for D_A = 0.1
df1 = pd.DataFrame({'ratioA': ratio_Adom, 'size_Adom': size_Adom})
df2 = pd.DataFrame({'ratioP': ratio_P, 'size_P': size_P})
df3 = pd.DataFrame({'ratioP': ratio_Pdom, 'size_Pdom': size_Pdom})
df4 = pd.DataFrame({'ratioA': ratio_A, 'size_A': size_A})
# Collect all ratios/sizes in dataframes for D_A = 0
df5 = pd.DataFrame({'ratioA': ratio_Adom0, 'size_Adom': size_Adom0})
df6 = pd.DataFrame({'ratioP': ratio_P0, 'size_P': size_P0})
df7 = pd.DataFrame({'ratioP': ratio_Pdom0, 'size_Pdom': size_Pdom0})
df8 = pd.DataFrame({'ratioA': ratio_A0, 'size_A': size_A0})
# Plot both parameter sets in the same phase diagram
plt.plot(df3.groupby('size_Pdom').max())
plt.plot(df4.groupby('size_A').min())
plt.plot(df1.groupby('size_Adom').min())
plt.plot(df2.groupby('size_P').max())
plt.plot(df5.groupby('size_Adom').min())
plt.plot(df6.groupby('size_P').max())
plt.plot(df7.groupby('size_Pdom').max())
plt.plot(df8.groupby('size_A').min())

plt.xlabel('system size [um]')
plt.ylabel('A/P')
plt.show()

In [ ]:
# font = {'family' : 'sans-serif',
#         'weight' : 'normal',
#         'size'   : 18}
# plt.rc('font', **font)
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')
F = df2.groupby('size_P').max()
S = df4.groupby('size_A').min()
topx = F.index.values
top = [x[0] for x in F.as_matrix()]
botx = S.index.values
bot = [x[0] for x in S.as_matrix()]
both_x = np.linspace(15, 54, 100)
top = np.interp(both_x, topx, top)
bot = np.interp(both_x, botx, bot)
plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 1.5, alpha=0.6, color='salmon')
# plt.plot(df1.groupby('size_Adom').min(), 'darkorange', linewidth=1.0)
# plt.plot(df4.groupby('size_A').min(), 'darkorange', linewidth=1.0)

plt.ylim(0.6, 1.1)
plt.xlim(15, 50)
ax = plt.gca()
ax.text(18, 1, r'\bf{Unpolarized (slow wins)}')
ax.text(40, 0.92, r'\bf{Polarized}')
ax.text(25, 0.73, r'\bf{Unpolarized (fast wins)}')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Da_Dp_dosage.pdf', transparent=True)
plt.show()

In [ ]:
extreme cases of cross talk
no cross talk no diffusion in one species
Finish this nb
Sample random

**Case: $D_a$ = 0**

In [ ]:
ratio_Adom0 = []
size_Adom0 = []
ratio_P0 = []
size_P0 = []
for file in glob.glob("/Volumes/Transcend/PARmodellingData/pickled_FindA051217/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        ratio_Adom0.append(p.ratio)
        size_Adom0.append(p.sys_size)
    elif (p.finished_in_time == 0) or (p.finished_in_time == 2):
        ratio_P0.append(p.ratio)
        size_P0.append(p.sys_size)

In [ ]:
ratio_Pdom0 = []
size_Pdom0 = []
ratio_A0 = []
size_A0 = []
for file in glob.glob("/Volumes/Transcend/PARmodellingData/pickled_FindP051217/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 0):
        ratio_Pdom0.append(p.ratio)
        size_Pdom0.append(p.sys_size)
    elif (p.finished_in_time == 1) or (p.finished_in_time == 2):
        ratio_A0.append(p.ratio)
        size_A0.append(p.sys_size)

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w', edgecolor='k')
df5 = pd.DataFrame({'ratioA': ratio_Adom0, 'size_Adom': size_Adom0})
df6 = pd.DataFrame({'ratioP': ratio_P0, 'size_P': size_P0})
plt.plot(df5.groupby('size_Adom').min())
plt.plot(df6.groupby('size_P').max())
df7 = pd.DataFrame({'ratioP': ratio_Pdom0, 'size_Pdom': size_Pdom0})
df8 = pd.DataFrame({'ratioA': ratio_A0, 'size_A': size_A0})
plt.plot(df7.groupby('size_Pdom').max())
plt.plot(df8.groupby('size_A').min())
plt.xlabel('system size [um]')
plt.ylabel('A/P ratio')
plt.show()

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')
F = df6.groupby('size_P').max()
S = df8.groupby('size_A').min()

topx = F.index.values
top = [x[0] for x in F.as_matrix()]
botx = S.index.values
bot = [x[0] for x in S.as_matrix()]
both_x = np.linspace(15, 54, 100)
top = np.interp(both_x, topx, top)
bot = np.interp(both_x, botx, bot)

plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 1.5, alpha=0.6, color='salmon')
# plt.plot(df5.groupby('size_Adom').min(), 'k', linewidth=1.0)
# plt.plot(df8.groupby('size_A').min(), 'k', linewidth=1.0)

plt.ylim(0.6, 1.1)
plt.xlim(15, 50)
ax = plt.gca()
ax.text(18, 1, r'\bf{Unpolarized (slow wins)}')
ax.text(40, 0.92, r'\bf{Polarized}')
ax.text(30, 0.7, r'\bf{Unpolarized (fast wins)}')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Da_Dp_dosage_0diff.pdf', transparent=True)
plt.show()


### Run for many sizes and many D combinations

In [ ]:
ratio_AdomM = []
size_AdomM = []
dA_AdomM = []
ratio_PM = []
size_PM = []
dA_PM = []
for file in glob.glob("/Users/hubatsl/Desktop/LongVsShortL/*.pickle"):
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        ratio_AdomM.append(p.ratio)
        size_AdomM.append(p.sys_size)
        dA_AdomM.append(p.dA)
    elif (p.finished_in_time == 0) or (p.finished_in_time == 2):
        ratio_PM.append(p.ratio)
        size_PM.append(p.sys_size)
        dA_PM.append(p.dA)

In [ ]:
ratio_PdomM = []
size_PdomM = []
dA_PdomM = []
ratio_AM = []
size_AM = []
dA_AM = []
for file in glob.glob("/Users/hubatsl/Desktop/FindP/*.pickle"):
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 0):
        ratio_PdomM.append(p.ratio)
        size_PdomM.append(p.sys_size)
        dA_PdomM.append(p.dA)
    elif (p.finished_in_time == 1) or (p.finished_in_time == 2):
        ratio_AM.append(p.ratio)
        size_AM.append(p.sys_size)
        dA_AM.append(p.dA)

In [ ]:
df5 = pd.DataFrame({'ratioA': ratio_AdomM, 'size_Adom': size_AdomM, 'dA_Adom': dA_AdomM})
df6 = pd.DataFrame({'ratioP': ratio_PM, 'size_P': size_PM, 'dA_PM': dA_PM})
df7 = pd.DataFrame({'ratioP': ratio_PdomM, 'size_Pdom': size_PdomM, 'dA_Pdom': dA_PdomM})
df8 = pd.DataFrame({'ratioA': ratio_AM, 'size_A': size_AM, 'dA_AM': dA_AM})
di = []
for D in df6['dA_PM'].unique():
    dfplot1 = df5[df5['dA_Adom'] == D]
    dfplot2 = df6[df6['dA_PM'] == D]
    plt.plot(dfplot1.groupby('size_Adom')['ratioA'].min())
    plt.plot(dfplot2.groupby('size_P')['ratioP'].max(), 'b')
    print(D)
    dfplot3 = df7[df7['dA_Pdom'] == D]
    dfplot4 = df8[df8['dA_AM'] == D]
    plt.plot(dfplot3.groupby('size_Pdom')['ratioP'].max())
    plt.plot(dfplot4.groupby('size_A')['ratioA'].min(), 'r')
    plt.xlabel('system size [um]')
    plt.ylabel('A/P ratio [um]')
    plt.show()
    a = dfplot4.groupby('size_A')['ratioA'].min()
    b = dfplot2.groupby('size_P')['ratioP'].max()
    c = (b-a).interpolate()
    di.append(c[c<0.03].index.max())
    plt.plot(c)
    plt.show()

In [ ]:
def make_graph_pretty(xlab, ylab, loc=0):
    color1 = 'black'#'darkorange'
    color2 = (0, 0, 0)
    color2 = (1, 1, 1)
    ax = plt.gca()
    ax.tick_params(axis='x', colors=color1, which='both')
    ax.tick_params(axis='y', colors=color1, which='both')
    plt.xlabel(xlab, color=color1)
    plt.ylabel(ylab, color=color1)
    ax.spines['left'].set_color(color1)
    ax.spines['bottom'].set_color(color1)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_facecolor(color2)
    plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
    plt.legend(frameon=False, loc=loc)

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w', edgecolor='k')
c1 = (132/255, 197/255, 97/255)
c2 = (223/255, 184/255, 88/255)
plt.plot(di, df6['dA_PM'].unique()/0.3, '.', label='PAR system', color=c1)
plt.plot(np.linspace(0, max(di), 20), np.linspace(0, 1, 20), '--', label='Perfect scaling', color=c2)
plt.xlim(0, None)
plt.ylim(0, None)
make_graph_pretty(r'system size', r'$\frac{D_A}{0.3\; \mu m^2/s}$')
plt.gcf().subplots_adjust(bottom=0.17, left=0.17)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Breakdown_vs_Dratio.pdf', transparent=True)
plt.show()

**Case: $D_a==D_p==0.3$**

In [ ]:
ratio_Adom0 = []
size_Adom0 = []
ratio_P0 = []
size_P0 = []
for file in glob.glob("/Users/hubatsl/Desktop/SymmetricSystemDosageFindA/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 1):
        ratio_Adom0.append(p.ratio)
        size_Adom0.append(p.sys_size)
    elif (p.finished_in_time == 0) or (p.finished_in_time == 2):
        ratio_P0.append(p.ratio)
        size_P0.append(p.sys_size)

In [ ]:
ratio_Pdom0 = []
size_Pdom0 = []
ratio_A0 = []
size_A0 = []
for file in glob.glob("/Users/hubatsl/Desktop/SymmetricSystemDosageFindP/*.pickle"):
#     print(file)/
    with open(file, 'rb') as f:
        p = pickle.load(f)
    if (p.finished_in_time == 0):
        ratio_Pdom0.append(p.ratio)
        size_Pdom0.append(p.sys_size)
    elif (p.finished_in_time == 1) or (p.finished_in_time == 2):
        ratio_A0.append(p.ratio)
        size_A0.append(p.sys_size)

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w', edgecolor='k')
df5 = pd.DataFrame({'ratioA': ratio_Adom0, 'size_Adom': size_Adom0})
df6 = pd.DataFrame({'ratioP': ratio_P0, 'size_P': size_P0})
plt.plot(df5.groupby('size_Adom').min())
plt.plot(df6.groupby('size_P').max())
df7 = pd.DataFrame({'ratioP': ratio_Pdom0, 'size_Pdom': size_Pdom0})
df8 = pd.DataFrame({'ratioA': ratio_A0, 'size_A': size_A0})
plt.plot(df7.groupby('size_Pdom').max())
plt.plot(df8.groupby('size_A').min())
plt.xlabel('system size [um]')
plt.ylabel('A/P ratio')
plt.show()

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')
F = df6.groupby('size_P').max()
S = df8.groupby('size_A').min()

topx = F.index.values
top = [x[0] for x in F.as_matrix()]
botx = S.index.values
bot = [x[0] for x in S.as_matrix()]
both_x = np.linspace(15, 54, 100)
top = np.interp(both_x, topx, top)
bot = np.interp(both_x, botx, bot)

plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 1.5, alpha=0.6, color='salmon')
# plt.plot(df5.groupby('size_Adom').min(), 'k', linewidth=1.0)
# plt.plot(df8.groupby('size_A').min(), 'k', linewidth=1.0)

plt.ylim(0.8, 1.2)
plt.xlim(15, 50)
ax = plt.gca()
ax.text(18, 1.15, r'\bf{Unpolarized (A wins)}')
ax.text(40, 0.99, r'\bf{Polarized}')
ax.text(18, 0.85, r'\bf{Unpolarized (P wins)}')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
plt.savefig('/Users/hubatsl/Dropbox (Lars DMS)/LRI/PAR_Size/Theory/Da_Dp_dosage_symm.pdf', transparent=True)
plt.show()

### Run different D ratios vs CPSS without cluster

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.3, 'dP': 0.3,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 25}

# In principle break down can happen due to dosage or due to diffusion asymmetry.
# It is unclear, whether these both act on the same time scale, or whether at some point,
# when diffusion is very similar, at ratios close to 1, the dosage ratio is dominant.
# In order to avoid getting two possible diffusion coefficients at the same size, diffusion and
# dosage should point in the same direction: if A is slowed down (which make P win)

sizes = np.linspace(10.3, 17.8, 8)
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]

s_above = [0.3 for x in siz]
s_below = [0 for x in siz]
s = [0.15 for x in siz]

for j in range(6):
    for i in range(len(siz)):
        siz[i]['dP'] = s[i]
        print(siz[i]['dP'])
    sims = Sim_Container(siz, no_workers=8)
    sims.init_simus()
    sims.run_simus()
    
    for i in range(len(siz)):
        outcome = sims.simus[i].finished_in_time
        if (outcome == 1) or (outcome == 0):
            s_above[i] = s[i]
            s[i] = s[i] - (s[i] - s_below[i])/2
        elif outcome == 2:
            s_below[i] = s[i]
            s[i] = s[i] + (s_above[i] - s[i])/2


In [ ]:
s=[0.00234375,
 0.00234375,
 0.01171875,
 0.02578125,
 0.04453125,
 0.07734374999999999,
 0.13359374999999998,
 0.29765624999999996]

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.3, 'dP': 0.3,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.00,
             'sys_size': 25}

sizes = np.linspace(10.3, 17.8, 8)
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]

s_above = [0.3 for x in siz]
s_below = [0 for x in siz]
s = [0.15 for x in siz]

for j in range(6):
    for i in range(len(siz)):
        siz[i]['dP'] = s[i]
        print(siz[i]['dP'])
    sims = Sim_Container(siz, no_workers=8)
    sims.init_simus()
    sims.run_simus()
    
    for i in range(len(siz)):
        outcome = sims.simus[i].finished_in_time
        if (outcome == 1) or (outcome == 0):
            s_above[i] = s[i]
            s[i] = s[i] - (s[i] - s_below[i])/2
        elif outcome == 2:
            s_below[i] = s[i]
            s[i] = s[i] + (s_above[i] - s[i])/2

In [ ]:
sizes = np.linspace(10.3, 17.8, 8)
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]

s_above = [0.3 for x in siz]
s_below = s
s1 = [i+0.01 for i in s]

for j in range(6):
    for i in range(len(siz)):
        siz[i]['dP'] = s1[i]
        print(siz[i]['dP'])
    sims1 = Sim_Container(siz, no_workers=8)
    sims1.init_simus()
    sims1.run_simus()
    
    for i in range(len(siz)):
        outcome = sims1.simus[i].finished_in_time
        if (outcome == 1) or (outcome == 0):
            s_below[i] = s1[i]
            s1[i] = s1[i] + (s_above[i] - s1[i])/2
        elif outcome == 2:
            s_above[i] = s1[i]
            s1[i] = s1[i] - (s1[i] - s_below[i])/2

In [ ]:
symmetric = {'alpha': 2, 'beta': 2, 'dA': 0.28, 'dP': 0.3,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.01,
             'sys_size': 25}
sizes = np.linspace(17.0, 18, 8)
siz = [copy.deepcopy(symmetric) for _ in sizes]
for i, dic in enumerate(siz):
    siz[i]['sys_size'] = sizes[i]

sims1 = Sim_Container(siz, no_workers=8)
sims1.init_simus()
sims1.run_simus()

## Redo symmetric and asymmetric dosage ratio diagrams, using MacBook instead of Cluster

In [ ]:
from ParDetAsymm import Dosage_CPSS
# Da = 0, Dp = 0.1
sz = [4, 15]
dc = Dosage_CPSS(5, 0.1, 8, sz, max_iter=10)
dc.simulate(dom='P')
dc.simulate(dom='A')
plt.plot(np.linspace(sz[0], sz[1], 8), dc.ratioP)
plt.plot(np.linspace(sz[0], sz[1], 8), dc.ratioA)
plt.show()

In [ ]:
# Da = 0.1, Dp = 1.1
sz = [10, 25]
dc = Dosage_CPSS(5, 0.1, 8, sz, max_iter=10)
dc.simulate(dom='P')
dc.simulate(dom='A')
plt.plot(np.linspace(sz[0], sz[1], 8), dc.ratioP)
plt.plot(np.linspace(sz[0], sz[1], 8), dc.ratioA)
plt.show()

In [ ]:
from ParDetAsymm import Dosage_CPSS
# Da = 0.1, Dp = 0.1
for da in np.linspace(0.0, 0.1, 16):
    print(da) 
    sz = [3.5, 10]
    base_set = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.0,
             'sys_size': 30}
    base_set['dA'] = da
    dc = Dosage_CPSS(5, 0.05, 24, sz, base_set=base_set, max_iter=16)
    dc.simulate(dom='P')
    dc.simulate(dom='A')
    plt.plot(np.linspace(sz[0], sz[1], 24), dc.ratioP)
    plt.plot(np.linspace(sz[0], sz[1], 24), dc.ratioA)
    plt.show()
    dc.pickle_data('0407_Dp01_Da' + str(da))

In [ ]:
dc.pickle_data('Dp01_Da' + str(da))

In [ ]:
dc.time/3600

In [ ]:
def pl(n):    
    with open(n, 'rb') as f:
            p = pickle.load(f)
            ax = plt.plot(np.linspace(p.sz_range[0], p.sz_range[1], p.precision), p.ratioP)
            plt.plot(np.linspace(p.sz_range[0], p.sz_range[1], p.precision), p.ratioA, ax[0].get_color())
pl('Dp01_Da0.0.pickle')
for da in np.linspace(0.02, 0.1, 7):
    pl('Dp01_Da' + str(da) + '.pickle')
plt.show()

In [ ]:
def pl(n):    
    with open(n, 'rb') as f:
            p = pickle.load(f)
            ax = plt.plot(np.linspace(p.sz_range[0], p.sz_range[1], p.precision), p.ratioP)
            plt.plot(np.linspace(p.sz_range[0], p.sz_range[1], p.precision), p.ratioA, ax[0].get_color())

for da in np.linspace(0.0, 0.1, 16):
    pl('0107_SimusCPSS/0107_Dp01_Da' + str(da) + '.pickle')
plt.show()

In [ ]:
def get_size(n, thresh):

    with open(n, 'rb') as f:
        p = pickle.load(f)    
        sz_interp = np.linspace(p.sz_range[0], p.sz_range[1], 5*p.precision)
        sz = np.linspace(p.sz_range[0], p.sz_range[1], p.precision)

        ind = np.argmin(abs(np.interp(sz_interp, sz,
                                      np.array(p.ratioA)-np.array(p.ratioP))-thresh))
    return sz_interp[ind]

In [ ]:
cpss01 = [get_size('Dp01_Da0.0.pickle', 0.01)]
cpss05 = [get_size('Dp01_Da0.0.pickle', 0.05)]
cpss1 = [get_size('Dp01_Da0.0.pickle', 0.1)]
d = [0]
for da in np.linspace(0.02, 0.1, 7):
    cpss01.append(get_size('Dp01_Da' + str(da) + '.pickle', 0.01))
    cpss05.append(get_size('Dp01_Da' + str(da) + '.pickle', 0.05))
    cpss1.append(get_size('Dp01_Da' + str(da) + '.pickle', 0.1))
    d.append(da/0.1)
plt.plot(d, cpss01, 'o', label='0.01')
plt.plot(d, cpss05, 'o', label='0.05')
plt.plot(d, cpss1, 'o', label='0.1')
plt.legend()
plt.xlim(0, 1.1)
plt.ylim(0, 11)
plt.xlabel('Da/0.1')
plt.ylabel('CPSS')
plt.show()

In [ ]:
cpss01 = []
cpss05 = []
cpss1 = []
d = []
for da in np.linspace(0.0, 0.1, 16):
    cpss01.append(get_size('0107_SimusCPSS/0107_Dp01_Da' + str(da) + '.pickle', 0.01))
    cpss05.append(get_size('0107_SimusCPSS/0107_Dp01_Da' + str(da) + '.pickle', 0.05))
    cpss1.append(get_size('0107_SimusCPSS/0107_Dp01_Da' + str(da) + '.pickle', 0.08))
    d.append(da/0.1)
plt.plot(d, cpss01, 'o', label='0.01')
plt.plot(d, cpss05, 'o', label='0.05')
plt.plot(d, cpss1, 'o', label='0.08')
plt.legend()
plt.xlim(0, 1.1)
plt.ylim(0, 11)
plt.xlabel('Da/0.1')
plt.ylabel('CPSS')
plt.show()

In [ ]:
cpss01_2000s = []
cpss05 = []
cpss1 = []
d = []
for da in np.linspace(0.0, 0.1, 16):
    cpss01_2000s.append(get_size('0407_2000sSimusCPSS/0407_Dp01_Da' + str(round(da,13)) + '.pickle', 0.01))
    cpss05.append(get_size('0407_2000sSimusCPSS/0407_Dp01_Da' + str(round(da,13)) + '.pickle', 0.05))
    cpss1.append(get_size('0407_2000sSimusCPSS/0407_Dp01_Da' + str(round(da,13)) + '.pickle', 0.08))
    d.append(da/0.1)
plt.plot(d, cpss01_2000s, 'o', label='0.01')
plt.plot(d, cpss05, 'o', label='0.05')
plt.plot(d, cpss1, 'o', label='0.08')
plt.legend()
plt.xlim(0, 1.1)
plt.ylim(0, 11)
plt.xlabel('Da/0.1')
plt.ylabel('CPSS')
plt.show()

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w', edgecolor='k')
c1 = (132/255, 197/255, 97/255)
c2 = (223/255, 184/255, 88/255)
# plt.plot(d, cpss01, '-', color=c1)
plt.plot(d, cpss01_2000s, 'x', label='PAR system 2000 s', color=c1)
# plt.plot(d, cpss01, '.', label='PAR system 1000 s', color=c1)
plt.plot(np.linspace(0, 1, 20), np.linspace(0, max(cpss01_2000s), 20), '--', label='Perfect scaling', color=c2)
plt.xlim(0, None)
plt.ylim(0, None)
make_graph_pretty(r'$\frac{D_A}{0.1\; \mu m^2/s}$', r'system size')
plt.gcf().subplots_adjust(bottom=0.17, left=0.17)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Breakdown_vs_DratioInvert.pdf', transparent=True)
plt.show()

In [ ]:
from ParDetAsymm import Dosage_CPSS
# Da = 0.1, Dp = 0.1
prec = 8
for da in np.linspace(0, 0, 1):
    print(da) 
    sz = [15, 60]
    base_set = {'alpha': 2, 'beta': 2, 'dA': 0.0, 'dP': 0.1,
             'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
             'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.0,
             'sys_size': 30}
    base_set['dA'] = da
    dc = Dosage_CPSS(5, 0.05, prec, sz, base_set=base_set, max_iter=16)
    dc.simulate(dom='P')
    dc.simulate(dom='A')
    plt.plot(np.linspace(sz[0], sz[1], prec), dc.ratioP)
    plt.plot(np.linspace(sz[0], sz[1], prec), dc.ratioA)
    plt.show()
    dc.pickle_data('2407_Dp01_Da' + str(da))

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')
with open('2307_Dp01_Da0.1.pickle', 'rb') as f:
    dc = pickle.load(f)

top = dc.ratioA
bot = dc.ratioP
both_x = np.linspace(dc.sz_range[0], dc.sz_range[1], dc.precision)

plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 1.5, alpha=0.6, color='salmon')

plt.ylim(0.8, 1.25)
plt.xlim(both_x[0], both_x[-1])
ax = plt.gca()
ax.text(5, 1.15, 'Unpolarized (A wins)')
ax.text(13.5, 0.99, 'Polarized', weight='bold')
ax.text(5, 0.85, 'Unpolarized (P wins)')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage_symm1.pdf', transparent=True)
plt.show()

In [ ]:
with open('2307_Dp01_Da0.0.pickle', 'rb') as f:
    dc = pickle.load(f)
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')


top = dc.ratioA
bot = dc.ratioP
both_x = np.linspace(dc.sz_range[0], dc.sz_range[1], dc.precision)

plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 2, alpha=0.6, color='salmon')

plt.ylim(0.3, 2)
plt.xlim(both_x[0], both_x[-1])
ax = plt.gca()
ax.text(5, 1.6, 'Unpolarized (A wins)')
ax.text(13.5, 0.99, 'Polarized', weight='bold')
ax.text(5, 0.45, 'Unpolarized (P wins)')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
# plt.savefig('/Users/lhcge/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage1.pdf', transparent=True)
plt.show()

In [ ]:
with open('2307_Dp01_Da0.0.pickle', 'rb') as f:
    dc = pickle.load(f)

In [ ]:
dc.simus.simus[0].T

In [ ]:
# Redo for t = 10000
from ParDetAsymm import Dosage_CPSS
base_set = {'alpha': 2, 'beta': 2, 'dA': 0.1, 'dP': 0.1,
         'kAP': 1, 'kPA': 1, 'koffA': 0.005, 'koffP': 0.005,
         'konA': 0.006, 'konP': 0.006, 'Ptot': 1, 'ratio': 1.0,
         'sys_size': 30}
sz = [4, 22]
dc = Dosage_CPSS(1.2, 0.8, 16, sz, base_set=base_set, max_iter=16)
dc.simulate(dom='P')
dc.simulate(dom='A')

In [ ]:
prec = 16
plt.plot(np.linspace(sz[0], sz[1], prec), dc.ratioP)
plt.plot(np.linspace(sz[0], sz[1], prec), dc.ratioA)
plt.show()
dc.pickle_data('0312_Dp01_Da01')

In [ ]:
fig=plt.figure(figsize=(3.75, 3.1), dpi= 300, facecolor='w')
with open('0312_Dp01_Da01.pickle', 'rb') as f:
    dc = pickle.load(f)

top = dc.ratioA
bot = dc.ratioP
both_x = np.linspace(dc.sz_range[0], dc.sz_range[1], dc.precision)

plt.fill_between(both_x, top, bot, alpha=1, color='grey')
plt.fill_between(both_x, bot, 0, alpha=0.6, color='cornflowerblue')
plt.fill_between(both_x, top, 1.5, alpha=0.6, color='salmon')

plt.ylim(0.8, 1.25)
plt.xlim(both_x[0], both_x[-1])
ax = plt.gca()
ax.text(5, 1.15, 'Unpolarized (A wins)')
ax.text(13.5, 0.99, 'Polarized', weight='bold')
ax.text(5, 0.85, 'Unpolarized (P wins)')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage_symm_10000s.pdf', transparent=True)
plt.show()

## Dosage for Wave Pinning (imported from Matlab)

In [ ]:
t = np.loadtxt('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/' +
               'WP_matlab_sizeDosageTop_10000.csv', delimiter=',');
b = np.loadtxt('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/' +
               'WP_matlab_sizeDosageBot_10000.csv', delimiter=',');

In [ ]:
plt.fill_between([4, 20], 0, 40, alpha=0.6, color='salmon')
plt.fill_between(t[:, 0], t[:, 1], np.interp(t[:, 0], np.flip(b[:, 0]), np.flip(b[:, 1])), alpha=1, color='grey')
plt.rcParams.update({'font.size': 14})
plt.ylim(0.9, 1.3)
plt.xlim(4, 20)
ax = plt.gca()
ax.text(6, 1.25, 'Unpolarized')
ax.text(12, 1.05, 'Polarized')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
make_graph_pretty(r'system size [$\mu m$]', r'A/P ratio')
plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage_WP.pdf', transparent=True)
plt.show()

## Dosage for Goryachev (imported from Matlab)

In [ ]:
b = np.loadtxt('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/' +
               'goryachev_matlab_sizeDosageTopBot.csv', delimiter=',');

In [ ]:
b_x = b[:, 1]
b_bot = b[:, 0][b[:, -1].astype(int)]
b_top = b[:, 0][b[:, -2].astype(int)-1]
first_ind = np.min(np.where(np.diff(b_bot-b_top)>0))+1
plt.fill_between(b_x, 0, 100, alpha=0.6, color='salmon')
plt.fill_between(b_x[first_ind:], b_top[first_ind:], b_bot[first_ind:], alpha=1, color='grey')

plt.ylim(0.0, 60)
ax = plt.gca()
ax.text(13, 50, 'Unpolarized', weight='bold')
ax.text(26, 18, 'Polarized', weight='bold')
plt.xlabel(r'system size [$\mu m$]')
plt.ylabel(r'A/P ratio')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
# plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage_GOR.pdf', transparent=True)
plt.show()

## Dosage for Otsuji (imported from Matlab)

In [ ]:
t = np.loadtxt('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/' +
               'otsuji_matlab_sizeDosageTop_10000.csv', delimiter=',');
b = np.loadtxt('/Users/hubatsch/Desktop/PARmodelling/MatlabDet/ActivatorInhibitor/' +
               'otsuji_matlab_sizeDosageBot_10000.csv', delimiter=',');

In [ ]:
plt.fill_between([4, 20], 0, 40, alpha=0.6, color='salmon')
plt.fill_between(t[:, 0], t[:, 1], np.interp(t[:, 0], np.flip(b[:, 0]), np.flip(b[:, 1])), alpha=1, color='grey')
plt.rcParams.update({'font.size': 14})
plt.ylim(0.0, 20)
plt.xlim(4, 20)
ax = plt.gca()
ax.text(14, 5, 'Polarized')
ax.text(6.5, 15, 'Unpolarized')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.gcf().subplots_adjust(bottom=0.17, left=0.15)
make_graph_pretty(r'system size [$\mu m$]', r'A/P ratio')
plt.savefig('/Users/hubatsch/Dropbox (Lars DMS)/LRI/PAR_Size/PAR_Size_MS_Data/Data_Figure3_Theory2/Da_Dp_dosage_OT.pdf', transparent=True)
plt.show()